In [4]:
import numpy as np
import torch

class Neuron:
    def __init__(self, id):
        self.id = id
        self.incoming_connections = []
        self.output = 0.0

class Connection:
    def __init__(self, in_node, out_node, weight, enabled=True):
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight
        self.enabled = enabled

class Genome:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        self.neurons = []
        self.connections = []
        self.fitness = 0.0

    def add_connection(self, in_node, out_node, weight):
        connection = Connection(in_node, out_node, weight)
        self.connections.append(connection)

    def add_neuron(self, neuron):
        self.neurons.append(neuron)

class NEAT:
    def __init__(self, population_size, input_size, output_size):
        self.population_size = population_size
        self.input_size = input_size
        self.output_size = output_size
        self.population = []

        for _ in range(population_size):
            genome = Genome(input_size, output_size)
            self.create_initial_connections(genome)
            self.population.append(genome)

    def create_initial_connections(self, genome):
        for i in range(genome.input_size):
            for j in range(genome.input_size, genome.input_size + genome.output_size):
                weight = np.random.uniform(-1.0, 1.0)
                genome.add_connection(i, j, weight)

    def feed_forward(self, genome, inputs):
        neurons = [Neuron(i) for i in range(genome.input_size + genome.output_size)]
        for i in range(genome.input_size):
            neurons[i].output = inputs[i]

        for connection in genome.connections:
            if connection.enabled:
                neurons[connection.out_node].incoming_connections.append(connection)

        for neuron in neurons[genome.input_size:]:
            total_input = 0.0
            for connection in neuron.incoming_connections:
                in_node = neurons[connection.in_node]
                total_input += in_node.output * connection.weight
            neuron.output = self.activate(total_input)

        outputs = [neurons[i].output for i in range(genome.input_size, genome.input_size + genome.output_size)]
        return outputs

    def activate(self, x):
        return torch.tanh(torch.tensor(x))

    def mutate_weights(self, genome):
        for connection in genome.connections:
            if np.random.uniform() < 0.1:
                connection.weight += np.random.uniform(-1.0, 1.0)

    def crossover(self, parent1, parent2):
        child = Genome(self.input_size, self.output_size)
        enabled_connections = []

        for connection1 in parent1.connections:
            enabled = False
            for connection2 in parent2.connections:
                if connection1.in_node == connection2.in_node and connection1.out_node == connection2.out_node:
                    enabled = connection1.enabled or connection2.enabled
                    break

            if enabled or np.random.uniform() < 0.5:
                child.add_connection(connection1.in_node, connection1.out_node, connection1.weight)
                enabled_connections.append((connection1.in_node, connection1.out_node))

        for connection2 in parent2.connections:
            if (connection2.in_node, connection2.out_node) not in enabled_connections:
                child.add_connection(connection2.in_node, connection2.out_node, connection2.weight)

        return child

    def mutate(self, genome):
        if np.random.uniform() < 0.05:
            self.mutate_weights(genome)

    def generate_offspring(self, parent1, parent2):
        child = self.crossover(parent1, parent2)
        self.mutate(child)
        return child

    def select_parent(self):
        fitnesses = [genome.fitness for genome in self.population]
        probabilities = np.exp(fitnesses) / np.sum(np.exp(fitnesses))
        return np.random.choice(self.population, p=probabilities)

    def evolve(self, generations):
        for _ in range(generations):
            offspring = []
            for _ in range(self.population_size):
                parent1 = self.select_parent()
                parent2 = self.select_parent()
                child = self.generate_offspring(parent1, parent2)
                offspring.append(child)

            self.population = offspring

    def evaluate_fitness(self, fitness_fn):
        for genome in self.population:
            inputs = np.random.uniform(-1.0, 1.0, self.input_size)
            outputs = self.feed_forward(genome, inputs)
            genome.fitness = fitness_fn(inputs, outputs)

def fitness_function(inputs, outputs):
    # Calculate the sum of output values
    output_sum = sum(outputs)

    # Return the fitness value (in this case, the sum of outputs)
    return output_sum


In [7]:
# Define the parameters
population_size = 50
input_size = 10
output_size = 5
generations = 150

# Create an instance of NEAT
neat = NEAT(population_size, input_size, output_size)

# Evolve the population
neat.evolve(generations)

# Evaluate fitness for the final population
neat.evaluate_fitness(fitness_function)

# Access the best genome
best_genome = max(neat.population, key=lambda x: x.fitness)
best_fitness = best_genome.fitness

# Print the best fitness value
print("Best Fitness:", best_fitness)


Best Fitness: tensor(4.4142, dtype=torch.float64)
